# Задача перевода

### Вспомогательные функции

PONS Online Dictionary API

In [103]:
API_KEY_PONS = '0d66df58d098cfeddd85278e673d49498f239569f9feb6d3356d40d773d3024d'

In [116]:
import requests

def translate_text_with_pons(term, source_language='en', target_language='ru'):
    api_key = API_KEY_PONS
    
    endpoint = 'https://api.pons.com/v1/dictionary'
    
    headers = {
        'X-Secret': api_key,
    }

    params = {
        'in': 'en',
        'language': 'ru',
        'l': f'{source_language}{target_language}',
         'q': term,
    }

    response = requests.get(endpoint, headers=headers, params=params)

    if response.status_code == 200:
        data = response.json()[0]
        
        translations = data.get('hits', [])
        if translations:
            try:
                translation = translations[0]['roms'][0]['arabs'][0]['translations'][0]['target']
                return translation
            except:
                return f'No translation found for: {term}\n'
        else:
            print(f'No translation found for: {term}\n')
    else:
        print(f'Error {response.status_code}: {response.text}\n')


terms_to_translate = ['servitude', 'liberty', 'sovereignty', 'security of person']
for term in terms_to_translate:
    print(f'{term} переводится как', end=' ')
    translated = translate_text_with_pons(term)
    if translated:
        print(f'{translated.split("<")[0]}')


servitude переводится как ра́бство 
liberty переводится как свобо́да 
sovereignty переводится как суверените́т 
security of person переводится как Error 204: 



Reverso API https://context.reverso.net/

In [117]:
!pip install reverso-api

In [142]:
from reverso_api.context import ReversoContextAPI
def get_translation(source_text, top_k = 3, source_lang='en', target_lang='ru'):
    api = ReversoContextAPI(
        source_text=source_text,
        target_text='',
        source_lang=source_lang,
        target_lang=target_lang)

    translations = []
    for source_word, translation, frequency, part_of_speech, inflected_forms in api.get_translations():
        translations.append((translation, frequency))
        if len(translations) == top_k:
            break
    if len(translations) == 0:
        return None
    translations.sort(key=lambda x: -x[1])
    translations, _ = zip(*translations)
    return translations

In [82]:
def get_contexts(source_text, top_k = 3, source_lang='en', target_lang='ru'):
    api = ReversoContextAPI(
        source_text=source_text,
        target_text='',
        source_lang=source_lang,
        target_lang=target_lang)
    
    contexts = []
    for source, target in api.get_examples():
        contexts.append(source.text + '==' + target.text)
        if len(contexts) == top_k:
            break
    return '\n'.join(contexts)

In [54]:
trans1 = get_translation('security of person', top_k = 3)
trans1

('личную неприкосновенность', 'личную безопасность', 'безопасность личности')

Функции для создания нового промпта из ключевых слов и исходного промпта

In [143]:
def make_prompt(source_prompt, key_words, top_k_translations=3, use_context=False, top_k_context=5):
    promt = 'Here are some ground truth information about translations:\n'
    for key_word in key_words:
        translations = get_translation(key_word, top_k_translations)
        if translations is None:
            continue
        promt_part = f'{key_word} translates as '
        for translation in translations:
            promt_part += translation + ' \ '
            
        promt += promt_part[:-3] + ';\n'
        
        if use_context==True:
            context = get_contexts(key_word, top_k_context)
            promt += f'Possible contexts for {key_word}:\n' + context + '.\n'
        
        
        
    return promt[:-2] + '.\nBased on provided evidence above, ' + source_prompt

In [93]:
# Плохо работает, не используется
def make_prompt_with_detection(source_prompt, key_words, **kwargs):
    promt = 'Here are some ground truth information about translations:\n'
    for key_word in key_words:
        translations = get_translation(key_word, **kwargs)
        promt_part = f'{key_word} translates as '
        for translation in translations:
            promt_part += translation + ' \ '
            
        promt += promt_part[:-3] + ';\n'
        
    return promt[:-2] + '.\nBased on provided evidence above, tell me if your translation has mistakes. If so, ' + source_prompt

### Алгоритм починки

**Шаг 1 - Написание промпта**.

Q: Translate the next paragraphs into Russian. Be as precise in terminology as possible.
Everyone has the right to life, liberty and security of person

A: Каждый человек имеет право на жизнь, свободу и безопасность личности.

**Шаг 2 - Выделение важных фраз (концепций)**.

Q: Identify all the important keyphrases from the above sentence in english and return a comma separated list in quotes

or

Q: Identify all the important keyphrases from the above sentence in english and return a comma separated list in quotes. Keyphrases should contain just a few words

or

Q: Identify all the important keyphrases from the above sentence in english and return a comma separated list, every keyphrases in quotes. Keyphrases should contain just a few words

A: right to life, liberty, security of person

**Шаг 3 - Поиск релевантных знаний в интернете и создание соответствующего промпта (а именно перевод ключевых фраз)**

Шаг предполагает использование написанной функции `make_prompt(source_prompt, key_words, top_k_translations=3, use_context=False, top_k_context=5)`

Q: Here are some ground truth information about translations:
right to life translates as право на жизнь;
liberty translates as свобода;
security of person translates as личную неприкосновенность.
Based on provided evidence above, 
Translate the next paragraphs into Russian. Be as precise in terminology as possible.
Everyone has the right to life, liberty and security of person

A: Каждый человек имеет право на жизнь, свободу и личную неприкосновенность

### Примеры шага 3

In [95]:
source_prompt = '''
Translate the next paragraphs into Russian. Be as precise in terminology as possible.
Everyone has the right to life, liberty and security of person
'''

key_words = ['right to life', 'liberty', 'security of person']

In [96]:
print(make_prompt(source_prompt, key_words, top_k_translations=1))

Here are some ground truth information about translations:
right to life translates as право на жизнь;
liberty translates as свобода;
security of person translates as личную неприкосновенность.
Based on provided evidence above, 
Translate the next paragraphs into Russian. Be as precise in terminology as possible.
Everyone has the right to life, liberty and security of person



In [97]:
print(make_prompt(source_prompt, key_words, top_k_translations=1, use_context=True, top_k_context=3))

Here are some ground truth information about translations:
right to life translates as право на жизнь;
Possible contexts for right to life:
Gender-biased legal systems also jeopardize women's right to life.==Правовые системы, поощряющие предвзятое отношение к женщинам, также ставят под угрозу их право на жизнь.
Innocent people had an equal right to life.==Ни в чем не повинные люди имеют такое же право на жизнь.
Otherwise, she has no right to life.==А, следовательно, у нее нет и права на жизнь..
liberty translates as свобода;
Possible contexts for liberty:
Capitalism and liberty are concepts that cannot be untangled.==Капитализм и свобода - понятия, которые не могут существовать отдельно.
Only liberty can ensure individual development and diversity.==Только свобода может гарантировать человеку его индивидуальное развитие и разнообразие.
It says that all anarchists want liberty.==Оно говорит о том, что все анархисты хотят свободы..
security of person translates as личную неприкосновеннос

In [98]:
print(make_prompt_with_detection(source_prompt, key_words, top_k=1))

Here are some ground truth information about translations:
right to life translates as право на жизнь;
liberty translates as свобода;
security of person translates as личную неприкосновенность.
Based on provided evidence above, tell me if your translation has mistakes. If so, 
Translate the next paragraphs into Russian. Be as precise in terminology as possible.
Everyone has the right to life, liberty and security of person



### Chatgpt fix

#### 1. Everyone is entitled to all the rights

In [100]:
source_prompt = '''
Translate the next paragraphs into Russian. Be as precise in terminology as possible.
Everyone is entitled to all the rights and freedoms set forth in this Declaration, without distinction of any kind, such as race, colour, sex, language, religion, political or other opinion, national or social origin, property, birth or other status. Furthermore, no distinction shall be made on the basis of the political, jurisdictional or international status of the country or territory to which a person belongs, whether it be independent, trust, non-self-governing or under any other limitation of sovereignty.
'''
key_words = ["every person", "right to all rights and freedoms", "this Declaration", "distinction of any kind", "race, color, sex, language, religion, political or other opinion, national or social origin, property, birth or other status", "no distinction shall be made", "political, jurisdictional or international status of the country or territory", "independent, trust, non-self-governing or under any other limitation of sovereignty"]

In [111]:
print(make_prompt(source_prompt, key_words, top_k_translations=1))

Here are some ground truth information about translations:
every person translates as каждый человек;
this Declaration translates as настоящей Декларации;
distinction of any kind translates as какой-либо дискриминации;
no distinction shall be made translates as должно проводиться никакого различия;
political, jurisdictional or international status of the country or territory translates as политического, правового или международного статуса страны или территории;
independent, trust, non-self-governing or under any other limitation of sovereignty translates as независимой, подопечной, несамоуправляющейся или как-либо иначе ограниченной.
Based on provided evidence above, 
Translate the next paragraphs into Russian. Be as precise in terminology as possible.
Everyone is entitled to all the rights and freedoms set forth in this Declaration, without distinction of any kind, such as race, colour, sex, language, religion, political or other opinion, national or social origin, property, birth or

#### 2. Everyone has the right to life, liberty and security of person

In [112]:
source_prompt = '''
Translate the next paragraphs into Russian. Be as precise in terminology as possible. Everyone has the right to life, liberty and security of person.
'''
key_words = ["right to life", "liberty", "security of person"]

In [113]:
print(make_prompt(source_prompt, key_words, top_k_translations=1))

Here are some ground truth information about translations:
right to life translates as право на жизнь;
liberty translates as свобода;
security of person translates as личную неприкосновенность.
Based on provided evidence above, 
Translate the next paragraphs into Russian. Be as precise in terminology as possible. Everyone has the right to life, liberty and security of person.



#### 3. No one shall be held in slavery or servitude; slavery and the slave trade shall be prohibited in all their forms

In [114]:
source_prompt = '''
Translate the next paragraphs into Russian. Be as precise in terminology as possible.
No one shall be held in slavery or servitude; slavery and the slave trade shall be prohibited in all their forms.
'''
key_words = ["held in slavery or servitude", "slavery and the slave trade", "prohibited in all their forms"]

In [115]:
print(make_prompt(source_prompt, key_words, top_k_translations=1))

Here are some ground truth information about translations:
held in slavery or servitude translates as содержаться в рабстве или в подневольном состоянии;
slavery and the slave trade translates as рабства и работорговли;
prohibited in all their forms translates as запрещаются во всех их видах.
Based on provided evidence above, 
Translate the next paragraphs into Russian. Be as precise in terminology as possible.
No one shall be held in slavery or servitude; slavery and the slave trade shall be prohibited in all their forms.



#### 4. Everyone has the right to recognition everywhere as a person before the law.

In [116]:
source_prompt = '''
Translate the next paragraphs into Russian. Be as precise in terminology as possible.
Everyone has the right to recognition everywhere as a person before the law.
'''
key_words = ["right to recognition", "everywhere as a person", "before the law"]

In [119]:
key_words = ["right to recognition everywhere", "as a person before the law"]

In [121]:
print(make_prompt(source_prompt, key_words, top_k_translations=2, use_context=True, top_k_context=2))

Here are some ground truth information about translations:
as a person before the law translates as его правосубъектности \ Правосубъектность;
Possible contexts for as a person before the law:
Report on birth registration and the right of everyone to recognition everywhere as a person before the law (June 2014)==Доклад по вопросу о регистрации рождения и праве каждого человека на повсеместное признание его правосубъектности (июнь 2014 г.)
5.3 The Committee notes that the author's claim under article 16 has not been substantiated, as there is no information on file which suggests that the author's son was denied recognition as a person before the law.==5.3 Комитет отмечает, что жалоба автора сообщения в соответствии со статьей 16 не обоснована, поскольку в материалах дела не имеется информации, которая указывала бы на то, что сыну автора сообщения было отказано в признании его правосубъектности..
Based on provided evidence above, 
Translate the next paragraphs into Russian. Be as precis

#### 5. Everyone has the right to an effective remedy by the competent national tribunals for acts violating the fundamental rights granted him by the constitution or by law.

In [122]:
source_prompt = '''
Translate the next paragraphs into Russian. Be as precise in terminology as possible.
Everyone has the right to an effective remedy by the competent national tribunals for acts violating the fundamental rights granted him by the constitution or by law.
'''
key_words = ["right to an effective remedy", "competent national tribunals", "acts violating the fundamental rights", "granted him by the constitution or by law"]

In [126]:
print(make_prompt(source_prompt, key_words, top_k_translations=2, use_context=True, top_k_context=2))

Here are some ground truth information about translations:
right to an effective remedy translates as право на эффективную правовую защиту \ права на эффективную правовую защиту;
Possible contexts for right to an effective remedy:
It might be useful for the Committee to understand how the right to an effective remedy functioned in the case law of other treaty bodies.==Комитету было бы целесообразно разобраться, каким образом право на эффективную правовую защиту действует в прецедентном праве других договорных органов.
As a general rule, the right to an effective remedy is recognized in Burkina Faso.==В целом в Буркина-Фасо признано право на эффективную правовую защиту..
competent national tribunals translates as компетентные национальные суды \ компетентных национальных судов;
Possible contexts for competent national tribunals:
Availability of effective protection and remedies in competent national tribunals to guarantee fundamental freedoms==Обеспечение эффективной защиты и средств за

#### 6. "1. Everyone has the right to freedom of peaceful assembly and association. 2. No one may be compelled to belong to an association."

In [136]:
source_prompt = '''
Translate the next paragraphs into Russian. Be as precise in terminology as possible.
1. Everyone has the right to freedom of peaceful assembly and association.
2. No one may be compelled to belong to an association.
'''
key_words = ["right to freedom of peaceful assembly", "right to freedom of association", "compelled to belong", "association"]

In [137]:
print(make_prompt(source_prompt, key_words, top_k_translations=1))

Here are some ground truth information about translations:
right to freedom of association translates as права на свободу ассоциации;
compelled to belong translates as принужден к вступлению;
association translates as ассоциация.
Based on provided evidence above, 
Translate the next paragraphs into Russian. Be as precise in terminology as possible.
1. Everyone has the right to freedom of peaceful assembly and association.
2. No one may be compelled to belong to an association.



#### 7. Everyone has the right to education....

In [132]:
source_prompt = '''
Translate the next paragraphs into Russian. Be as precise in terminology as possible.
1. Everyone has the right to education. Education shall be free, at least in the elementary and fundamental stages. Elementary education shall be compulsory. Technical and professional education shall be made generally available and higher education shall be equally accessible to all on the basis of merit.
2. Education shall be directed to the full development of the human personality and to the strengthening of respect for human rights and fundamental freedoms. It shall promote understanding, tolerance and friendship among all nations, racial or religious groups, and shall further the activities of the United Nations for the maintenance of peace.
3. Parents have a prior right to choose the kind of education that shall be given to their children.
'''
key_words = ["right to education", "free", "elementary and fundamental stages", "compulsory", "technical and professional education", "higher education", "equally accessible", "basis of merit"
"directed to the full development of the human personality", "strengthening of respect for human rights and fundamental freedoms", "promote understanding, tolerance, and friendship", "United Nations", "maintenance of peace"
"Parents have a prior right", "choose the kind of education", "given to their children"]

In [133]:
print(make_prompt(source_prompt, key_words, top_k_translations=2, use_context=True, top_k_context=2))

Here are some ground truth information about translations:
right to education translates as права на образование \ право на образование;
Possible contexts for right to education:
Thus, the right to education in socialist countries where...==Так, право на образование в социалистических странах, в которых провозглашается единство...
The second-ranking ethnic group not realizing their right to education were the Vlachs.==На втором месте среди этнических групп, не реализующих свое право на образование, находились влахи..
free translates as свободный \ бесплатный;
Possible contexts for free:
Linux is free software (free meaning freedom).==Linux - это свободное (т.е. бесплатное) программное обеспечение.
Other categories include free, mostly free, moderately free and repressed.==В нем есть свободные страны, в основном свободные, умеренно свободные, подавленные..
compulsory translates as обязательный \ принудительный;
Possible contexts for compulsory:
In some countries preferences are voluntar

#### 8. Everyone has the right freely to participate

In [138]:
source_prompt = '''
Translate the next paragraphs into Russian. Be as precise in terminology as possible.
1. Everyone has the right freely to participate in the cultural life of the community, to enjoy the arts and to share in scientific advancement and its benefits.
2. Everyone has the right to the protection of the moral and material interests resulting from any scientific, literary or artistic production of which he is the author.
'''
key_words = ["right to freely participate", "cultural life of the community", "enjoy the arts", "share in scientific advancement", "benefits", "protection of moral and material interests", "scientific, literary or artistic production", "author"]

In [141]:
print(make_prompt(source_prompt, key_words, top_k_translations=1, use_context=True, top_k_context=1))

Here are some ground truth information about translations:
right to freely participate translates as право на свободное участие;
Possible contexts for right to freely participate:
Everyone shall have the right to freely participate and organise peaceful, unarmed assemblies.==Каждый имеет право на свободное участие и организацию мирных собраний без оружия..
cultural life of the community translates as культурной жизни общества;
Possible contexts for cultural life of the community:
We believe in everyone's right to participate in the cultural life of the community and to enjoy the arts.==Каждый имеет право на свободу участвовать в культурной жизни общества, наслаждаться искусством..
enjoy the arts translates as наслаждаться искусством;
Possible contexts for enjoy the arts:
We believe in everyone's right to participate in the cultural life of the community and to enjoy the arts.==Каждый имеет право на свободу участвовать в культурной жизни общества, наслаждаться искусством..
share in scie

### Claude fix

#### 1. Everyone has the right to an effective remedy by the competent national tribunals for acts violating the fundamental rights granted him by the constitution or by law.

In [152]:
source_prompt = '''
Translate the next paragraphs into Russian. Be as precise in terminology as possible.
Everyone has the right to an effective remedy by the competent national tribunals for acts violating the fundamental rights granted him by the constitution or by law.
'''
key_words = ["right to an effective remedy", "competent national tribunals", "acts violating the fundamental rights", 
             "rights granted by the constitution or by law"]

In [157]:
print(make_prompt(source_prompt, key_words, top_k_translations=2, use_context=True, top_k_context=3))

Here are some ground truth information about translations:
right to an effective remedy translates as право на эффективную правовую защиту \ права на эффективную правовую защиту;
Possible contexts for right to an effective remedy:
It might be useful for the Committee to understand how the right to an effective remedy functioned in the case law of other treaty bodies.==Комитету было бы целесообразно разобраться, каким образом право на эффективную правовую защиту действует в прецедентном праве других договорных органов.
As a general rule, the right to an effective remedy is recognized in Burkina Faso.==В целом в Буркина-Фасо признано право на эффективную правовую защиту.
Member States shall provide for reasonable ï time limits and other necessary rules for the applicant to exercise his/her right to an effective remedy pursuant to paragraph 1.==Государства-члены устанавливают предельные сроки и другие необходимые правила для осуществления заявителем права на эффективную правовую защиту во

#### 2. Everyone has the right freely to participate

In [158]:
source_prompt = '''
Translate the next paragraphs into Russian. Be as precise in terminology as possible.
1. Everyone has the right freely to participate in the cultural life of the community, to enjoy the arts and to share in scientific advancement and its benefits.
2. Everyone has the right to the protection of the moral and material interests resulting from any scientific, literary or artistic production of which he is the author.
'''
key_words = ["right to participate","cultural life","enjoy the arts","share in scientific advancement","protection of moral interests","protection of material interests","scientific production","literary production","artistic production","author"]

In [160]:
print(make_prompt(source_prompt, key_words, top_k_translations=2, use_context=True, top_k_context=1))

Here are some ground truth information about translations:
right to participate translates as право участвовать \ право на участие;
Possible contexts for right to participate:
Everyone has the right to participate in health-related decision-making that affects them.==Каждый человек имеет право участвовать в процессе принятия решений, касающихся вопросов здравоохранения и затрагивающих его..
cultural life translates as культурной жизни \ культурную жизнь;
Possible contexts for cultural life:
Every cultural sector contributes to the citizens' access to cultural life.==Кроме того, каждый сектор культуры франкоговорящего сообщества способствует в целом тому, чтобы граждане имели доступ к культурной жизни..
enjoy the arts translates as наслаждаться искусством \ насладитесь искусством;
Possible contexts for enjoy the arts:
We believe in everyone's right to participate in the cultural life of the community and to enjoy the arts.==Каждый имеет право на свободу участвовать в культурной жизни об

### Gigachat fix

#### 1. All human beings are born free and equal in dignity and rights

In [161]:
source_prompt = '''
Translate the next paragraphs into Russian. Be as precise in terminology as possible.
All human beings are born free and equal in dignity and rights. They are endowed with reason and conscience and should act towards one another in a spirit of brotherhood.
'''
key_words = ["All human beings", "born free and equal", "dignity and rights", "endowed with reason and conscience", "should act towards one another", "spirit of brotherhood"]

In [163]:
print(make_prompt(source_prompt, key_words, top_k_translations=2, use_context=True, top_k_context=2))

Here are some ground truth information about translations:
All human beings translates as всех людей \ все люди;
Possible contexts for All human beings:
All human beings have the same brain structures and regions.==У всех людей одинаковые структуры мозга и регионы.
All human beings share the same universal needs and that behind every negative emotion lies an unmet universal need.==У всех людей есть одинаковые универсальные потребности и что за каждой негативной эмоцией скрывается неудовлетворенная универсальная потребность..
born free and equal translates as рождаются свободными и равными \ рождены свободными и равными;
Possible contexts for born free and equal:
All people are born free and equal the advantage and the rights.==Все люди рождаются свободными и равными в своем достоинстве и правах.
It sets the highest standards by asserting that all people are born free and equal in dignity and rights.==Она определяет самые высокие нормы, подтверждая, что все люди рождаются свободными и р

#### 2. Everyone is entitled to all the rights and

In [164]:
source_prompt = '''
Translate the next paragraphs into Russian. Be as precise in terminology as possible.
Everyone is entitled to all the rights and freedoms set forth in this Declaration, without distinction of any kind, such as race, colour, sex, language, religion, political or other opinion, national or social origin, property, birth or other status. Furthermore, no distinction shall be made on the basis of the political, jurisdictional or international status of the country or territory to which a person belongs, whether it be independent, trust, non-self-governing or under any other limitation of sovereignty.
'''
key_words = ["All human beings", "born free and equal", "dignity and rights", "endowed with reason and conscience", 
             "should act towards one another in a spirit of brotherhood", "Everyone is entitled to all the rights and freedoms set forth in this Declaration",
             "without distinction of any kind", "such as race colour sex language religion political or other opinion", 
             "national or social origin", "property birth or other status", 
             "no distinction shall be made on the basis of the political jurisdictional or international status of the country or territory to which a person belongs",
             "whether it be independent trust non-self-governing or under any other limitation of sovereignty"]

In [167]:
print(make_prompt(source_prompt, key_words, top_k_translations=2, use_context=True, top_k_context=1))

Here are some ground truth information about translations:
All human beings translates as всех людей \ все люди;
Possible contexts for All human beings:
All human beings have the same brain structures and regions.==У всех людей одинаковые структуры мозга и регионы..
born free and equal translates as рождаются свободными и равными \ рождены свободными и равными;
Possible contexts for born free and equal:
All people are born free and equal the advantage and the rights.==Все люди рождаются свободными и равными в своем достоинстве и правах..
dignity and rights translates as достоинства и прав \ достоинство и права;
Possible contexts for dignity and rights:
Everyone is equal in dignity and rights.==Мы все равны в том, что касается достоинства и прав..
endowed with reason and conscience translates as наделены разумом и совестью;
Possible contexts for endowed with reason and conscience:
All human beings are born free and equal. They are endowed with reason and conscience and should act toward

#### 3. Everyone has the right to life, liberty and security of person.

In [168]:
source_prompt = '''
Translate the next paragraphs into Russian. Be as precise in terminology as possible.
Everyone has the right to life, liberty and security of person.
'''
key_words = ["right to life", "liberty", "security of person"]

In [170]:
print(make_prompt(source_prompt, key_words, top_k_translations=1, use_context=True, top_k_context=1))

Here are some ground truth information about translations:
right to life translates as право на жизнь;
Possible contexts for right to life:
Gender-biased legal systems also jeopardize women's right to life.==Правовые системы, поощряющие предвзятое отношение к женщинам, также ставят под угрозу их право на жизнь..
liberty translates as свобода;
Possible contexts for liberty:
Capitalism and liberty are concepts that cannot be untangled.==Капитализм и свобода - понятия, которые не могут существовать отдельно..
security of person translates as личную неприкосновенность;
Possible contexts for security of person:
Concern was expressed over the right to security of person, particularly for indigenous inhabitants and illegal immigrants.==Также высказывается обеспокоенность по поводу права на личную неприкосновенность прежде всего в том, что касается коренных жителей и незаконных иммигрантов..
Based on provided evidence above, 
Translate the next paragraphs into Russian. Be as precise in termino

## And so on... Гигачад не работает :)

#### 4. No one shall be held in slavery or servitude

In [ ]:
source_prompt = '''
'''
key_words = []

In [ ]:
print(make_prompt(source_prompt, key_words, top_k_translations=1))

#### 5. Everyone has the right to recognition

In [ ]:
source_prompt = '''
'''
key_words = []

In [ ]:
print(make_prompt(source_prompt, key_words, top_k_translations=1))

#### 6. Everyone has the right to an effective remedy

In [ ]:
source_prompt = '''
'''
key_words = []

In [ ]:
print(make_prompt(source_prompt, key_words, top_k_translations=1))

#### 7. No one shall be subjected to arbitrary arrest

In [ ]:
source_prompt = '''
'''
key_words = []

In [ ]:
print(make_prompt(source_prompt, key_words, top_k_translations=1))

#### 8.  Everyone has the right to freedom of

In [ ]:
source_prompt = '''
'''
key_words = []

In [ ]:
print(make_prompt(source_prompt, key_words, top_k_translations=1))

#### 9. .Everyone has the right to take part in the

In [ ]:
source_prompt = '''
'''
key_words = []

In [ ]:
print(make_prompt(source_prompt, key_words, top_k_translations=1))

#### 10. Everyone, as a member of society, has the

In [ ]:
source_prompt = '''
'''
key_words = []

In [ ]:
print(make_prompt(source_prompt, key_words, top_k_translations=1))

#### 11. Everyone has the right to rest and leisure

In [ ]:
source_prompt = '''
'''
key_words = []

In [ ]:
print(make_prompt(source_prompt, key_words, top_k_translations=1))

#### 12. Everyone has the right freely to participate

In [ ]:
source_prompt = '''
'''
key_words = []

In [ ]:
print(make_prompt(source_prompt, key_words, top_k_translations=1))